In [92]:
import requests
from typing import List
import torch

d:\Transformer-From-Scratch\venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:283: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [8]:
response=requests.get("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt") 

In [9]:
response

<Response [200]>

In [75]:
text=response.text

In [52]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



### voacb/tokenization

We will build basic tokenization as of now which will be of character based tokenization


##### tiktoken

This also can be used to tokenize. I mean we don't have to worry about tokenization stuff

```py
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o") # this download the encoding used for gpt-4o model
enc.encode("hello world ")
```

In [62]:
sorted(set(text)) # we have all the characters here 

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [82]:
char_to_index={char:index for index,char in enumerate(sorted(set(text)))}
index_to_char={index:char for index,char in enumerate(sorted(set(text)))}

In [84]:
print(index_to_char[2])
print(char_to_index['!'])

!
2


In [91]:
word='hello world'
def encode(sent:str)->List[int]:
    outpur_seq=[]
    for char in sent:
        outpur_seq.append(char_to_index[char])
    return outpur_seq
    

def decode(seq:List[int])->str:
    output_seq=""
    for index in seq:
        output_seq+=index_to_char[index]
    
    return output_seq

print(encode(word))
print(decode([46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [93]:
# encode the text
data = torch.tensor(encode(text), dtype=torch.long)

In [95]:
data.shape

torch.Size([1115394])

In [96]:
data[:20]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56])

In [36]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o") # this download the encoding used for gpt-4o model
enc.encode("hello world ")

[24912, 2375, 220]

In [38]:
enc.encode('4bcgc4bb')

[19, 16715, 45670, 19, 10029]

In [47]:
for i in range(1500,1800):
    print(enc.decode([i]))

ful
 dif
til
na
 were
ہ
 ع
't
.h
####
ople
 into
 sur
ows
 Tr
amp
 De
air
 {

 cre
 make
rol
ina
от
ta
 א
(f
ու
�
SE
્
ے
 Con
ु
 result
_b
 rel
_id
ات
 loc
">

ics
')
 sub
error
ethod
 )
ze
 fl
ી
 Se
્�
af
ह
ho
�
 java
ody
 म
ml
ts
 list
�
ins
_d
ww
ת
En
ా
מ
�
 than
’t
ем
itle
ente
 first
ica
 aw
ę
 bo
By
 प
Se
–
Par
je
 import
 tem
Item
ു
]

 س
value
ious
work
ause
print
ка
lick
ule
 back
ense
 start
 ein
 only
(

 You
ures
 ke
ower
**

****************
arr
�
的
 its
raw
ร
 fil
 string
ces
 inform
ą
 form
ments
ү
urs
ities
ayer
 look
 let
 static
ember
day
 best
म
Value
ler
大
 under
ు
 trans
ender
 most
 find
 person
 ed
ustom
ла
].
 help
olog
ки
string
 Wh
 ng
ateg
lection
den
oth
 rem
ا�
κ
 people
 !=
 Al
أ
 car
।
const
ி
ọ
ാ
la
ಿ�
ед
ğ
 He
 ass
irect
 want
ari
://
ession
                           
ren
ան
 An
ै
ия
bo
={
ு
gram
.D
br
             
      
 don
 long
он
!


 gu
 sk
ern
Im
wa
22
 inst
load
rough
 um
�
?


ha
щ
 أ
ets
_C
 final
μ
40
 arr
 &&
ב
λ
时
 read
ë
ש
อ
orld
uc
];


Im not a writer and has nopt written anything before. So this content is raw with minimal amount of technical jargons and i tried to explain things in easiest possible way

# Self Attention

Why - 

What - 

How - 


See when we create vector embeddings using word2vec we get the static embeding means that embeding try to capture the avg meaning of the sentecnce. But it is not good way to create them. The most commong example for this i want to recite from the book called " Hands on largge langusga emodel by jay Alammar & Marrteen Grootendorst O'relly publication is river bank and money bank. So here bank means two diff things and if we use static embeding to represent the word that would lead to missunderstanding. So we have to solve this problem by generating dynamic embeding which can have the contect of surrounding words as well.


Question is how we do that- 

Simplest possible asnwer is take into account the static embedings of surrounding words or the words in that sequence. And by that we mean just have the weighted sum of all the embedings. by doing that we can can caputure the dynamic meaning of the word. 

But next quesion is what those weight will be like how can we calculate those weights. 

One thought could be why not calculating the similarity between the main word and the context/neighbour words. 

Lets try to implement this

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2") # this may occupy some space in local storage 


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 430.57it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


****************
So to implement this I'll be using  2 sentences

1. The fisherman sat quietly on the bank of the river watching the water flow. - rletaed to river bank 
2. She visited the bank to deposit her salary into her savings account - related to money bank

In [ ]:
import torch

In [ ]:
sentence1 = "River bank is calm"
sentence2 = "Bank stores money"

In [ ]:
# now lets calculate the embedings of each word first  
words1 = sentence1.split()
words2 = sentence2.split()

# Get embeddings for each word
embeddings1 = {word: torch.from_numpy(model.encode(word)) for word in words1} #  # I'll be using pytorch for mathemetical ops
embeddings2 = {word: torch.from_numpy(model.encode(word)) for word in words2}

In [76]:
# Now I'll calculate the dot product of all the embedings of first sentence with the word bank and get the score/weight 
weights1 = torch.tensor([
    torch.dot(embeddings1['bank'], embeddings1[word])
    for word in embeddings1
])

weights1

tensor([0.4996, 1.0000, 0.2495, 0.2183])

In [77]:
# To normalize the weights lets apply softmax
weights1 = torch.softmax(weights1, dim=0)
weights1

tensor([0.2391, 0.3943, 0.1862, 0.1804])

In [ ]:
# multiply these weights with embeding vectors of all the words in sentence 1
embedding_matrix = torch.stack([embeddings1[word] for word in embeddings1]) # 4*384 - coz we have 4 unique words in sent1

In [ ]:
e_bank1 = torch.matmul(weights1, embedding_matrix) # this is now new embeding of the word bank in context with sentence1

And now if we calculate the dot product of this with the word river

In [88]:
print("Old Similarity score between River and bank : ",torch.dot(embeddings1['River'],embeddings1['bank']))
print("new Similarity score between River and bank : ",torch.dot(embeddings1['River'],e_bank1))

Old Similarity score between River and bank :  tensor(0.4996)
new Similarity score between River and bank :  tensor(0.5381)


See we have increased the similarity between river and bank by doing this

Now lets do the same for second sentence

In [89]:
# Now I'll calculate the dot product of all the embedings of first sentence with the word bank and get the score/weight 
weights2 = torch.tensor([
    torch.dot(embeddings2['Bank'], embeddings2[word])
    for word in embeddings2
])
weights2 = torch.softmax(weights2, dim=0)

embedding_matrix_2 = torch.stack([embeddings2[word] for word in embeddings2]) # 12*384 - coz we have 12 unique words in sent1

e_bank2 = torch.matmul(weights2, embedding_matrix_2)

In [91]:
print("Old Similarity score between Money and bank : ",torch.dot(embeddings2['money'],embeddings2['Bank']))
print("new Similarity score between Money and bank : ",torch.dot(embeddings2['money'],e_bank2))

Old Similarity score between Money and bank :  tensor(0.4690)
new Similarity score between Money and bank :  tensor(0.5875)


Here also similarity is increased

### Issues with this approach